In [1]:
%defaultDatasource jdbc:h2:mem:db

In [2]:
DROP TABLE IF EXISTS Filme;
DROP TABLE IF EXISTS Avaliador;
DROP TABLE IF EXISTS Avaliacao;
DROP TABLE IF EXISTS Franquia;
DROP TABLE IF EXISTS FranquiaFilme;
DROP TABLE IF EXISTS Genero;
DROP TABLE IF EXISTS GeneroFilme;
DROP TABLE IF EXISTS Pessoa;
DROP TABLE IF EXISTS PessoaFilme;
DROP TABLE IF EXISTS Sequencia;
DROP TABLE IF EXISTS Streaming;
DROP TABLE IF EXISTS StreamingFilme;

In [3]:
CREATE TABLE Filme (
    id_TMDB INTEGER NOT NULL,
    id_IMDB VARCHAR(20) NOT NULL,
    titulo VARCHAR(200),
    titulo_original VARCHAR(200),
    sinopse VARCHAR(1000),
    duracao INTEGER,
    ano INTEGER,
    classificacao VARCHAR(50),
    situacao VARCHAR(20),
    idioma_original VARCHAR(5),
    orcamento FLOAT,
    receita FLOAT,
    num_oscars FLOAT,
    PRIMARY KEY(id_TMDB, id_IMDB)
) AS SELECT
    id_TMDB,
    id_IMDB,
    titulo,
    titulo_original,
    sinopse,
    duracao,
    ano,
    classificacao,
    situacao,
    idioma_original,
    orcamento,
    receita,
    num_oscars
FROM CSVREAD('CSV/Filme.csv');

In [4]:
CREATE TABLE Avaliador (
    id VARCHAR(10) NOT NULL,
    nome VARCHAR(10),
    PRIMARY KEY(id)
) AS SELECT
    id,
    nome
FROM CSVREAD('CSV/Avaliador.csv');

CREATE TABLE Avaliacao (
    id_avaliador VARCHAR(10) NOT NULL,
    id_filme INTEGER NOT NULL,
    nota FLOAT,
    PRIMARY KEY(id_avaliador, id_filme),
    FOREIGN KEY(id_avaliador)
      REFERENCES Avaliador(id)
        ON DELETE NO ACTION
        ON UPDATE NO ACTION,
    FOREIGN KEY(id_filme)
      REFERENCES Filme(id_TMDB)
        ON DELETE NO ACTION
        ON UPDATE NO ACTION
) AS SELECT
    id_avaliador,
    id_filme_TMDB,
    nota
FROM CSVREAD('CSV/Avaliacao.csv');

In [5]:
CREATE TABLE Franquia (
    nome VARCHAR(200) NOT NULL,
    PRIMARY KEY(nome)
) AS SELECT
    nome
FROM CSVREAD('CSV/Franquia.csv');

CREATE TABLE FranquiaFilme (
    nome_franquia VARCHAR(200) NOT NULL,
    id_filme INTEGER NOT NULL,
    PRIMARY KEY(nome_franquia, id_filme),
    FOREIGN KEY(nome_franquia)
      REFERENCES Franquia(nome)
        ON DELETE NO ACTION
        ON UPDATE NO ACTION,
    FOREIGN KEY(id_filme)
      REFERENCES Filme(id_TMDB)
        ON DELETE NO ACTION
        ON UPDATE NO ACTION
) AS SELECT
    nome_franquia,
    id_filme_TMDB
FROM CSVREAD('CSV/FranquiaFilme.csv');

CREATE TABLE Genero (
    nome VARCHAR(50) NOT NULL,
    PRIMARY KEY(nome)
) AS SELECT
    nome
FROM CSVREAD('CSV/Genero.csv');

CREATE TABLE GeneroFilme (
    nome_genero VARCHAR(50) NOT NULL,
    id_filme INTEGER NOT NULL,
    PRIMARY KEY(nome_genero, id_filme),
    FOREIGN KEY(nome_genero)
      REFERENCES Genero(nome)
        ON DELETE NO ACTION
        ON UPDATE NO ACTION,
    FOREIGN KEY(id_filme)
      REFERENCES Filme(id_TMDB)
        ON DELETE NO ACTION
        ON UPDATE NO ACTION
) AS SELECT
    nome_genero,
    id_filme_TMDB
FROM CSVREAD('CSV/GeneroFilme.csv');

In [6]:
CREATE TABLE Pessoa (
    id_TMDB INTEGER NOT NULL,
    id_IMDB VARCHAR(20),
    nome VARCHAR(200),
    nacionalidade VARCHAR(300),
    num_oscars INTEGER
) AS SELECT
    id_TMDB,
    id_IMDB,
    nome,
    nacionalidade,
    num_oscars
FROM CSVREAD('CSV/Pessoa.csv');

CREATE TABLE PessoaFilme (
    id_pessoa INTEGER NOT NULL,
    id_filme INTEGER NOT NULL,
    ator BOOL,
    diretor BOOL,
    roteirista BOOL,
    PRIMARY KEY(id_pessoa, id_filme),
    FOREIGN KEY(id_pessoa)
      REFERENCES Pessoa(id_TMDB)
        ON DELETE NO ACTION
        ON UPDATE NO ACTION,
    FOREIGN KEY(id_filme)
      REFERENCES Filme(id_TMDB)
        ON DELETE NO ACTION
        ON UPDATE NO ACTION
) AS SELECT
    id_pessoa_TMDB,
    id_filme_TMDB,
    ator,
    diretor,
    roteirista
FROM CSVREAD('CSV/PessoaFilme.csv');

In [7]:
CREATE TABLE Sequencia (
    id_filme INTEGER NOT NULL,
    id_filme_sequencia INTEGER NOT NULL,
    PRIMARY KEY(id_filme, id_filme_sequencia),
    FOREIGN KEY(id_filme)
      REFERENCES Filme(id_TMDB)
        ON DELETE NO ACTION
        ON UPDATE NO ACTION,
    FOREIGN KEY(id_filme_sequencia)
      REFERENCES Filme(id_TMDB)
        ON DELETE NO ACTION
        ON UPDATE NO ACTION
) AS SELECT
    id_filme_TMDB,
    id_filme_sequencia_TMDB
FROM CSVREAD('CSV/Sequencia.csv');

CREATE TABLE Streaming (
    nome VARCHAR(150) NOT NULL,
    PRIMARY KEY(nome)
) AS SELECT
    nome
FROM CSVREAD('CSV/Streaming.csv');

CREATE TABLE StreamingFilme (
    nome_streaming VARCHAR(150) NOT NULL,
    id_filme INTEGER NOT NULL,
    PRIMARY KEY(nome_streaming, id_filme),
    FOREIGN KEY(nome_streaming)
      REFERENCES Streaming(nome)
        ON DELETE NO ACTION
        ON UPDATE NO ACTION,
    FOREIGN KEY(id_filme)
      REFERENCES Filme(id_TMDB)
        ON DELETE NO ACTION
        ON UPDATE NO ACTION
) AS SELECT
    nome_streaming,
    id_filme_TMDB
FROM CSVREAD('CSV/StreamingFilme.csv');

In [8]:
SELECT * FROM Filme;
SELECT * FROM Avaliador;
SELECT * FROM Avaliacao;
SELECT * FROM Franquia;
SELECT * FROM FranquiaFilme;
SELECT * FROM Genero;
SELECT * FROM GeneroFilme;
SELECT * FROM Pessoa;
SELECT * FROM PessoaFilme;
SELECT * FROM Sequencia;
SELECT * FROM Streaming;
SELECT * FROM StreamingFilme;

# Queries no Dataset

In [9]:
/* Relação entre sucesso com o público (receita)
   e sucesso com a crítica (nota média) dos filmes */

DROP TABLE IF EXISTS FilmeReceitaNota;

CREATE VIEW FilmeReceitaNota AS
    SELECT A.id_filme, F.titulo, F.ano, F.receita, AVG(A.nota) nota_media
        FROM Avaliacao A, Filme F
        WHERE A.id_filme = F.id_TMDB
        GROUP BY A.id_filme;

-- Ordenação decrescente por receita
SELECT titulo, receita, nota_media 
    FROM FilmeReceitaNota
    ORDER BY receita DESC;

-- Ordenação decrescente por nota média
SELECT titulo, receita, nota_media 
    FROM FilmeReceitaNota
    ORDER BY nota_media DESC;

In [10]:
/* Gênero que fez mais sucesso com a crítica, isto é, 
   que tem os filmes com as melhores notas médias, na década de 2000 */

DROP TABLE IF EXISTS GeneroCritica;

CREATE VIEW GeneroCritica AS
    SELECT G.nome, SUM(F.nota_media) nota_total, COUNT(F.id_filme) qtdade_filme
        FROM Genero G, FilmeReceitaNota F, GeneroFilme GF
        WHERE G.nome = GF.nome_genero
              AND F.id_filme = GF.id_filme
              AND ((F.ano >= 2000) AND (F.ano < 2010))
        GROUP BY G.nome;

SELECT nome 
    FROM GeneroCritica
    WHERE (nota_total / qtdade_filme) = (SELECT MAX(nota_total / qtdade_filme)
                                             FROM GeneroCritica);

In [11]:
/* Gênero que fez mais sucesso o público, isto é, 
   que tem os filmes com a maior renda média, na década de 2000 */

DROP TABLE IF EXISTS GeneroPublico;

CREATE VIEW GeneroPublico AS
    SELECT G.nome, SUM(F.receita) receita_total, COUNT(F.id_filme) qtdade_filme
        FROM Genero G, FilmeReceitaNota F, GeneroFilme GF
        WHERE G.nome = GF.nome_genero
              AND F.id_filme = GF.id_filme
              AND ((F.ano >= 2000) AND (F.ano < 2010))
        GROUP BY G.nome;
        
SELECT nome
    FROM GeneroPublico
    WHERE (receita_total / qtdade_filme) = (SELECT MAX(receita_total / qtdade_filme)
                                              FROM GeneroPublico);

Ficção científica

In [12]:
/* Franquia que fez mais sucesso com a crítica,
   isto é, que tem os filmes com a maior nota média */

DROP TABLE IF EXISTS FranquiaCritica;

CREATE VIEW FranquiaCritica AS
    SELECT FR.nome, SUM(FI.nota_media) nota_total, COUNT(FF.id_filme) qtdade_filme
        FROM Franquia FR, FilmeReceitaNota FI, FranquiaFilme FF 
        WHERE FR.nome = FF.nome_franquia
              AND FI.id_filme = FF.id_filme
        GROUP BY FR.nome;
        
SELECT nome
    FROM FranquiaCritica
    WHERE (nota_total / qtdade_filme) = (SELECT MAX(nota_total / qtdade_filme)
                                             FROM FranquiaCritica);

The Lord of the Rings

In [13]:
/* Franquia que fez mais sucesso com o público,
   isto é, que tem os filmes com a maior renda média */

DROP TABLE IF EXISTS FranquiaPublico;

CREATE VIEW FranquiaPublico AS
    SELECT FR.nome, SUM(FI.receita) receita_total, COUNT(FF.id_filme) qtdade_filme
        FROM Franquia FR, Filme FI, FranquiaFilme FF 
        WHERE FR.nome = FF.nome_franquia
              AND FI.id_TMDB = FF.id_filme
        GROUP BY FR.nome;
        
SELECT nome
    FROM FranquiaPublico
    WHERE (receita_total / qtdade_filme) = (SELECT MAX(receita_total / qtdade_filme)
                                              FROM FranquiaPublico);

Avatar Collection